# Copy of Roberts approach in python to compute best emitting spo vs neutron output imaging time

- location of the MATLAB files: /Users/hkromer/02_PhD/02_Data/10.OutputVSEmittingSpot_Robert/


In [7]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from itertools import product
%matplotlib inline

## 1. Boundary conditions

- clarify the following with Robert:
    - units of FOV?
    - units of detector size?
    - what is the flux utilization?
    - imaging time raw?
    - imaging time effective>?
    - how does the imaging time have units seconds? Because of the magnification?

In [4]:
# model assumptions
d_so_min = 50 # source object minimum distance, mm
d_sd_min = 50 # source detector minimum distance, mm
d_od_min = 100 # object detector minimum distance, mm
d_sd_max = 10000 # source detector maximum distance, mm
d_so_max = 10000 # source object maximum distance, mm
FOV_min = 200 # field of view (degree) required to be satisfied, mm, max size the object can have in the cone
det_size_max = 1000 # maximum detector size, mm lenght of the detector
B_d = 0.8 # assumed detector resolution, mm
resolution_goal = 1.2 # target overall maximum allowed resolution, mm
detector_efficiency = 0.05 # assumed detector efficiency, [-]
detector_counts = 3000 # required flat field counts per pixel for an image, [-]
output_bias = 2 # forward bias of output, for DD reaction, approx. vs. isotropic

## 2. Range of parameters to calculate for

In [5]:
# range of parameters to calculate for
ni = 100 # number of intervals to sample in emitting spot and source strength

f_min = 0.5 # lower limit of emitting spot to use, mm
f_max = 26 # upper limit of emitting spot to use, mm
f_list = np.linspace(f_min, f_max, ni) # focal spot, mm, over a range of f_min to f_max

e_min = 7.5 # lower limit of neutron output 1eX n/s
e_max = 12.5 # upper limit of neutron output 1eX n/s
exp_list = np.linspace(e_min, e_max, ni) # use outputs of e_min to e_max in steps of ni
o_list = np.asarray([10**e for e in exp_list]) # calculates output (neutrons/s) based on given exponent

## 3. Function to get the optimum geometry

In [31]:
def get_magnification(row, d_so):
    d_sd = row.name
    return (d_sd/d_so)

ni = 50 # number of options in each d_sd and d_so
# d_so_s = pd.Series(np.linspace(d_so_min,d_so_max, ni), name='d_so')
d_so_s = pd.Series(np.arange(ni,10000+ni, ni), name='d_so') # Roberts data

ni = 50 # number of options in each d_sd and d_so
# d_sd_s = pd.Series(np.linspace(d_sd_min,d_sd_max, ni), name='d_sd')
d_sd_s = pd.Series(np.arange(ni,10000+ni, ni), name='d_sd')
df_distances = pd.DataFrame(index=d_so_s, columns=d_sd_s) # rows: source to object; cols: source to detector distance
M = df_distances.apply(lambda x: get_magnification(x, df_distances.index), axis=0) # dataframe with all the magnifications

def get_optimum_geometry(
    output, # calculates output (neutrons/s) based on given exponent (this is o_list)
    f, # focal spot, mm
    output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
    d_so_min, # source object minimum distance, mm
    d_od_min, # object detector minimum distance, mm
    d_sd_max, # source detector maximum distance, mm
    d_so_max, # source object maximum distance, mm
    df_distances, # rows: source to object; cols: source to detector distance
    M, # magnification
    FOV_min, # field of view required to be satisfied
    det_size_max, # maximum detector size
    B_d, # assumed detector resolution, mm
    resolution_goal, # target overall maximum allowed resolution, mm
    detector_efficiency, # assumed detector efficiency
    detector_counts):# required flat field counts per pixel for an image, [-]
    
#     # remove these
#     f = 3
#     output = o_list[8]



    
    B_do = M.copy().rdiv(B_d) # blur in detector scaled to object plane
    B_fo = (1-(M.copy().rdiv(1))).multiply(f)# blur in focal spot scaled to object plane
    B_o = np.sqrt(B_do.multiply(B_do) + B_fo.multiply(B_fo))# overall resolution (gaussian blurs B_do and B_fo assumed)


    
    # flux is not the same!
    flux_s = (output/4/np.pi/d_sd_s**2) * output_bias # flux at the detector
    flux_s.name = 'flux_at_detector'
    flux_utilization = M**2  # flux utilization?

    imaging_time_raw_s = detector_counts/flux_s/detector_efficiency
    imaging_time_raw_s.name ='raw_imaging_time'
    imaging_time_raw = pd.DataFrame([imaging_time_raw_s for ii in range(0,len(d_so_s))])
    imaging_time_raw = imaging_time_raw.set_index(d_so_s, drop=True)
    imaging_time_raw.columns = d_sd_s
#     print(imaging_time_raw)
#     print(flux_utilization)
    
#     def calc_effective_time(row, imaging_time_raw_s):
#         # multiply each element in the row with the respective element in imaging_time_raw_s
#     #     print(row)

#         return row * imaging_time_raw_s.values


#     df_imaging_time_effective = flux_utilization.apply(lambda x: calc_effective_time(x, imaging_time_raw_s), axis=1)
#     print(df_imaging_time_effective)

    df_imaging_time_effective = flux_utilization.multiply(imaging_time_raw)
#     print(df_imaging_time_effective)
    
    # zero out forbidden values
    # object to detector distance
#     def get_d_od(row, d_so):
#         d_sd = row.name
#         return (d_sd-d_so)

#     d_od = df_distances.apply(lambda x: get_d_od(x, df_distances.index), axis=0) # dataframe with all the magnifications

#     d_od = pd.DataFrame([d_sd_s for ii in range(0,len(d_so_s))]) - pd.DataFrame([d_so for ii in range(0,len(d_so_s))])
    df_d_sd = pd.DataFrame([d_sd_s for ii in range(0,len(d_so_s))])
    df_d_sd = df_d_sd.set_index(d_so_s, drop=True)
    df_d_sd.columns = d_sd_s

    df_d_so = pd.DataFrame([d_so_s for ii in range(0,len(d_sd_s))]).T
    df_d_so = df_d_so.set_index(d_so_s, drop=True)
    df_d_so.columns = d_sd_s

    d_od = df_d_sd - df_d_so

    start_time = time.time()
    M_max = det_size_max/FOV_min # maximum allowed magnification
    M_forbidden = M>M_max
    B_o_forbidden = B_o>resolution_goal
    d_od_forbidden = d_od<d_od_min   
    forbidden_mask = d_od_forbidden + M_forbidden + B_o_forbidden
    duration = (time.time() - start_time)
    print("--- %s seconds ---" % (duration))

#     print(forbidden_mask)
#     dfs = [B_o, flux_utilization, df_imaging_time_effective]
#     for df in dfs:
#         df[B_o_forbidden] = 0
#         df[M_forbidden] = 0
#         df[d_od_forbidden] = 0

    df_imaging_time_effective[forbidden_mask] = 0
    # set the zeros to ~inf to not mess up finding minimum
    # return df_imaging_time_effective

    df_imaging_time_effective[df_imaging_time_effective==0] = 1e10;
    best_imaging_time = np.min(np.min(df_imaging_time_effective))
    df_imaging_time_effective = df_imaging_time_effective[df_imaging_time_effective == best_imaging_time]
    best_d_sd = df_imaging_time_effective.idxmin().idxmin()
    best_d_so =  df_imaging_time_effective.loc[:,best_d_sd].idxmin()

    return pd.Series([best_imaging_time, best_d_so, best_d_sd], index=['best_imaging_time', 'best_d_so', 'best_d_sd'])
#     return pd.Series([best_imaging_time], index=['best_imaging_time'])

    # f: emitting spot size
    # best_imaging_time: in seconds
    # best_d_so: source to object distance in mm
    # best_d_sd: source to detector distance in mm
start_time = time.time()

a = get_optimum_geometry(
    3e15, # calculates output (neutrons/s) based on given exponent (this is o_list)
    20, # focal spot, mm
    output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
    d_so_min, # source object minimum distance, mm
    d_od_min, # object detector minimum distance, mm
    d_sd_max, # source detector maximum distance, mm
    d_so_max, # source object maximum distance, mm
    df_distances,
    M,
    FOV_min, # field of view required to be satisfied
    det_size_max, # maximum detector size
    B_d, # assumed detector resolution, mm
    resolution_goal, # target overall maximum allowed resolution, mm
    detector_efficiency, # assumed detector efficiency
    detector_counts)
duration = (time.time() - start_time)
print("--- %s seconds ---" % (duration))
a

--- 0.12648677825927734 seconds ---
--- 0.41860294342041016 seconds ---


best_imaging_time       0.000668
best_d_so            2100.000000
best_d_sd            2200.000000
dtype: float64

In [80]:
def get_magnification(row, d_so):
    d_sd = row.name
    return (d_sd/d_so)

ni = 5 # number of options in each d_sd and d_so
# d_so_s = pd.Series(np.linspace(d_so_min,d_so_max, ni), name='d_so')
d_so_ar = np.arange(ni,10000+ni, ni) # Roberts data
d_so_ar = np.linspace(d_so_min,d_so_max, ni) # Roberts data
d_so_s = pd.Series(d_so_ar, name='d_so')

ni = 10 # number of options in each d_sd and d_so
# d_sd_s = pd.Series(np.linspace(d_sd_min,d_sd_max, ni), name='d_sd')
d_sd_ar = np.arange(ni,10000+ni, ni) # Roberts data
d_sd_ar = np.linspace(d_sd_min,d_sd_max, ni) # Roberts data
d_sd_s = pd.Series(d_sd_ar, name='d_sd')

d_so_ndar = np.empty((d_so_ar.shape[0],d_sd_ar.shape[0]))
d_sd_ndar = np.empty((d_so_ar.shape[0],d_sd_ar.shape[0]))

d_so_ndar = np.tile(d_so_ar, (d_sd_ar.shape[0],1)).T
d_sd_ndar = np.tile(d_sd_ar, (d_so_ar.shape[0],1))


# magnification
M_ndar = np.empty((d_so_ar.shape[0],d_sd_ar.shape[0]))
M_ndar = d_sd_ndar/d_so_ndar

def numpy_get_optimum_geometry(
    output, # calculates output (neutrons/s) based on given exponent (this is o_list)
    f, # focal spot, mm
    output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
    d_so_min, # source object minimum distance, mm
    d_od_min, # object detector minimum distance, mm
    d_sd_max, # source detector maximum distance, mm
    d_so_max, # source object maximum distance, mm
    M_ndar, # magnification
    FOV_min, # field of view required to be satisfied
    det_size_max, # maximum detector size
    B_d, # assumed detector resolution, mm
    resolution_goal, # target overall maximum allowed resolution, mm
    detector_efficiency, # assumed detector efficiency
    detector_counts):# required flat field counts per pixel for an image, [-]
    

    B_do_nd = np.empty((M_ndar.shape[0],M_ndar.shape[1]))
    B_do_nd = np.true_divide(B_d, M_ndar)
    
    B_fo_nd = np.empty((M_ndar.shape[0],M_ndar.shape[1]))
    B_fo_nd = np.multiply
    B_fo = (1-(M.copy().rdiv(1))).multiply(f)# blur in focal spot scaled to object plane
    B_o = np.sqrt(B_do.multiply(B_do) + B_fo.multiply(B_fo))# overall resolution (gaussian blurs B_do and B_fo assumed)


    
    # flux is not the same!
    flux_s = (output/4/np.pi/d_sd_s**2) * output_bias # flux at the detector
    flux_s.name = 'flux_at_detector'
    flux_utilization = M**2  # flux utilization?

    imaging_time_raw_s = detector_counts/flux_s/detector_efficiency
    imaging_time_raw_s.name ='raw_imaging_time'
    imaging_time_raw = pd.DataFrame([imaging_time_raw_s for ii in range(0,len(d_so_s))])
    imaging_time_raw = imaging_time_raw.set_index(d_so_s, drop=True)
    imaging_time_raw.columns = d_sd_s
#     print(imaging_time_raw)
#     print(flux_utilization)
    
#     def calc_effective_time(row, imaging_time_raw_s):
#         # multiply each element in the row with the respective element in imaging_time_raw_s
#     #     print(row)

#         return row * imaging_time_raw_s.values


#     df_imaging_time_effective = flux_utilization.apply(lambda x: calc_effective_time(x, imaging_time_raw_s), axis=1)
#     print(df_imaging_time_effective)

    df_imaging_time_effective = flux_utilization.multiply(imaging_time_raw)
#     print(df_imaging_time_effective)
    
    # zero out forbidden values
    # object to detector distance
#     def get_d_od(row, d_so):
#         d_sd = row.name
#         return (d_sd-d_so)

#     d_od = df_distances.apply(lambda x: get_d_od(x, df_distances.index), axis=0) # dataframe with all the magnifications

#     d_od = pd.DataFrame([d_sd_s for ii in range(0,len(d_so_s))]) - pd.DataFrame([d_so for ii in range(0,len(d_so_s))])
    df_d_sd = pd.DataFrame([d_sd_s for ii in range(0,len(d_so_s))])
    df_d_sd = df_d_sd.set_index(d_so_s, drop=True)
    df_d_sd.columns = d_sd_s

    df_d_so = pd.DataFrame([d_so_s for ii in range(0,len(d_sd_s))]).T
    df_d_so = df_d_so.set_index(d_so_s, drop=True)
    df_d_so.columns = d_sd_s

    d_od = df_d_sd - df_d_so

    start_time = time.time()
    M_max = det_size_max/FOV_min # maximum allowed magnification
    M_forbidden = M>M_max
    B_o_forbidden = B_o>resolution_goal
    d_od_forbidden = d_od<d_od_min   
    forbidden_mask = d_od_forbidden + M_forbidden + B_o_forbidden
    duration = (time.time() - start_time)
    print("--- %s seconds ---" % (duration))

#     print(forbidden_mask)
#     dfs = [B_o, flux_utilization, df_imaging_time_effective]
#     for df in dfs:
#         df[B_o_forbidden] = 0
#         df[M_forbidden] = 0
#         df[d_od_forbidden] = 0

    df_imaging_time_effective[forbidden_mask] = 0
    # set the zeros to ~inf to not mess up finding minimum
    # return df_imaging_time_effective

    df_imaging_time_effective[df_imaging_time_effective==0] = 1e10;
    best_imaging_time = np.min(np.min(df_imaging_time_effective))
    df_imaging_time_effective = df_imaging_time_effective[df_imaging_time_effective == best_imaging_time]
    best_d_sd = df_imaging_time_effective.idxmin().idxmin()
    best_d_so =  df_imaging_time_effective.loc[:,best_d_sd].idxmin()

    return pd.Series([best_imaging_time, best_d_so, best_d_sd], index=['best_imaging_time', 'best_d_so', 'best_d_sd'])
#     return pd.Series([best_imaging_time], index=['best_imaging_time'])

          0          1          2          3          4           5  \
0  1.000000  23.111111  45.222222  67.333333  89.444444  111.555556   
1  0.019704   0.455391   0.891078   1.326765   1.762452    2.198139   
2  0.009950   0.229961   0.449972   0.669983   0.889994    1.110006   
3  0.006656   0.153818   0.300980   0.448142   0.595304    0.742466   
4  0.005000   0.115556   0.226111   0.336667   0.447222    0.557778   

            6           7           8           9  
0  133.666667  155.777778  177.888889  200.000000  
1    2.633826    3.069513    3.505200    3.940887  
2    1.330017    1.550028    1.770039    1.990050  
3    0.889628    1.036791    1.183953    1.331115  
4    0.668333    0.778889    0.889444    1.000000  


## 4. Test the function

In [24]:
from itertools import product
df_parameters = pd.DataFrame(
    {'f_list': f_list,
    'o_list': o_list,
    'exp_list': exp_list}
)
# df_parameters = df_parameters.iloc[0:50,:]
# print(len(df_parameters))
df_parameters = pd.DataFrame(list(product(df_parameters['f_list'], df_parameters['exp_list'])), columns=['f_list', 'exp_list'])
df_parameters['o_list'] = df_parameters['exp_list'].apply(lambda x: 10**(x))

print(df_parameters)



      f_list   exp_list        o_list
0        0.5   7.500000  3.162278e+07
1        0.5   7.550505  3.552262e+07
2        0.5   7.601010  3.990342e+07
3        0.5   7.651515  4.482447e+07
4        0.5   7.702020  5.035240e+07
...      ...        ...           ...
9995    26.0  12.297980  1.986003e+12
9996    26.0  12.348485  2.230924e+12
9997    26.0  12.398990  2.506051e+12
9998    26.0  12.449495  2.815107e+12
9999    26.0  12.500000  3.162278e+12

[10000 rows x 3 columns]


In [13]:
start_time = time.time()
df_out = pd.DataFrame()
for ii in range(0,5):
    # test for the first ten entries
    for jj in range(0,5):
        f = f_list[ii]
        output = o_list[jj]
        exp = exp_list[jj]
        optimums = get_optimum_geometry(
            output, # calculates output (neutrons/s) based on given exponent (this is o_list)
            f, # focal spot, mm
            output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
            d_so_min, # source object minimum distance, mm
            d_od_min, # object detector minimum distance, mm
            d_sd_max, # source detector maximum distance, mm
            d_so_max, # source object maximum distance, mm
            FOV_min, # field of view required to be satisfied
            det_size_max, # maximum detector size
            B_d, # assumed detector resolution, mm
            resolution_goal, # target overall maximum allowed resolution, mm
            detector_efficiency, # assumed detector efficiency
            detector_counts)# required flat field counts per pixel for an image, [-]
        optimums['exp'] = exp
        optimums['f'] = f
        optimums['output'] = output
        
        df_out = df_out.append(optimums, ignore_index=True)
        

print("--- %s seconds ---" % (time.time() - start_time))
df_out

--- 14.568666934967041 seconds ---


,best_d_sd,best_d_so,best_imaging_time,exp,f,output
0,200.0,100.0,1907.440947,7.500000,0.500000,3.162278e+07
1,200.0,100.0,1698.032706,7.550505,0.500000,3.552262e+07
2,200.0,100.0,1511.614330,7.601010,0.500000,3.990342e+07
3,200.0,100.0,1345.661880,7.651515,0.500000,4.482447e+07
4,200.0,100.0,1197.928506,7.702020,0.500000,5.035240e+07
5,200.0,100.0,1907.440947,7.500000,0.757576,3.162278e+07
6,200.0,100.0,1698.032706,7.550505,0.757576,3.552262e+07
7,200.0,100.0,1511.614330,7.601010,0.757576,3.990342e+07
8,200.0,100.0,1345.661880,7.651515,0.757576,4.482447e+07
9,200.0,100.0,1197.928506,7.702020,0.757576,5.035240e+07


In [ ]:
start_time = time.time()
df_parameters[['best_imaging_time', 'best_d_so', 'best_d_sd']] = df_parameters.apply(lambda x: get_optimum_geometry(
            x['o_list'], # calculates output (neutrons/s) based on given exponent (this is o_list)
            x['f_list'], # focal spot, mm
            output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
            d_so_min, # source object minimum distance, mm
            d_od_min, # object detector minimum distance, mm
            d_sd_max, # source detector maximum distance, mm
            d_so_max, # source object maximum distance, mm
            FOV_min, # field of view required to be satisfied
            det_size_max, # maximum detector size
            B_d, # assumed detector resolution, mm
            resolution_goal, # target overall maximum allowed resolution, mm
            detector_efficiency, # assumed detector efficiency
            detector_counts), axis=1)# required flat field counts per pixel for an image, [-])

print("--- %s seconds ---" % (time.time() - start_time))
df_parameters

In [ ]:
outfile = '/Users/hkromer/02_PhD/02_Data/10.OutputVSEmittingSpot_Robert/df_parameters_0to50.csv'
df_parameters.to_csv(outfile)

In [114]:
0.272727+0.018595

0.29132199999999997

In [23]:
df_parameters

,f_list,exp_list,o_list
0,0.500000,7.500000,3.162278e+07
1,0.500000,7.550505,3.552262e+07
2,0.500000,7.601010,3.990342e+07
3,0.500000,7.651515,4.482447e+07
4,0.500000,7.702020,5.035240e+07
...,...,...,...
2495,13.121212,9.772727,5.925531e+09
2496,13.121212,9.823232,6.656291e+09
2497,13.121212,9.873737,7.477172e+09
2498,13.121212,9.924242,8.399287e+09


In [34]:
jj = 0
jj % 100 == 0

True